In [2]:
import numpy as np

## Task 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [3]:
def subkey_generator_default(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[3]] 
    k_3 = [key[0],key[3],key[4],key[7]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[7]] 
    k_6 = [key[2],key[3],key[4],key[5]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter)

    return (w + k)%p

In [4]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])
    
    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()
    return w_array
        




In [5]:
def encryption(u,k,p,round,substitution,transposition,subkey_sum,linear, subkey_generator):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)         
        #print("post subkey_sum",v)
        y = substitution(v,p)
        #print("post timesTwo",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [6]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(x)


[4 0 0 9 7 0 0 3]


## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [7]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p

#Calcolo dell'inversa della matrice modulo p
def inv_matrix(m, p):
    # Calcolo del determinante della matrice
    det = int(np.round(np.linalg.det(m)))  # Determinante della matrice
    det_inv = pow(det, -1, p)  # Inverso del determinante modulo p

    # Calcolo della matrice aggiunta (trasposta della matrice dei cofattori)
    cofactors = np.zeros_like(m, dtype=int)
    n = m.shape[0]
    for i in range(n):
        for j in range(n):
            minor = np.delete(np.delete(m, i, axis=0), j, axis=1)  # Minore
            cofactor = ((-1) ** (i + j)) * int(np.round(np.linalg.det(minor)))
            cofactors[i, j] = cofactor

    adj_matrix = cofactors.T  # Trasposta della matrice dei cofattori

    # Calcolo dell'inversa modulo p
    m_inv = (det_inv * adj_matrix) % p

    return m_inv

def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    param_matrix_inv = inv_matrix(param_matrix, p)

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round):
    k_list = subkey_generator_default(k)

    for i in range(round,-1,-1):
        #print("round : ", i)
        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        #print("post subkey_sum",z)
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            #print("post linear", w)
        else:
            w = z
        y = inv_transposition(w)
        #print("post transpo",y)
        x = inv_timesTwo(y,p)
        #print("post timesTwo",x)
        
    return x

Test for inv_linear

In [8]:
a = linear(np.array([1,2,3,5,4,6,4,8]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[0 1 4 6 7 0 9 6]
[1 2 3 5 4 6 4 8]


In [9]:
u = decryption(x,k,p,r)
print(u)

[1 0 0 0 0 0 0 0]


Other tests for decryption

In [10]:
a = np.array([1,8,3,6,8,6,9,2]) #change plaintext to test
b = np.array([1,0,4,0,3,0,2,0]) #change key to test
c = encryption(a,b,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(c)
d = decryption(c,b,p,r)
print(d)

[ 8  6  1  8  6  8 10  3]
[1 8 3 6 8 6 9 2]


## Task 3
Identify the overall linear relationship for this simplified AES-like cipher, that is find the
matrices $A ∈ F^{(ℓ_x × ℓ_k)}$ and $B ∈ F^{(ℓ_x × ℓ_u)}$ such that
$$x = E(k, u) = Ak + Bu \mod p$$
with all operations in the field $\mathbb{F} = GF(p)$.

In [11]:
def linear_relationship():  #pass the encryption function as a parameter
    k_a = np.eye(8, dtype=int)
    e_a = np.zeros(8, dtype=int)
    A = np.empty((8,0), dtype=int)

    e_b = np.eye(8, dtype=int)
    k_b = np.zeros(8, dtype=int)
    B = np.empty((8,0), dtype=int)

    #k = e_j, u = 0
    for i in range(8):

        encrypted_value = encryption(e_a, np.asarray(k_a[i]), p, r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear,
                                    subkey_generator=subkey_generator_default)
        A = np.hstack((A, encrypted_value.reshape(-1, 1)))
        #print(encrypted_value)
        #print(k_a[i])
    #print(A)

    for i in range(8):

        encrypted_value = encryption(np.asarray(e_b[i]),k_b,p,r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear,
                                    subkey_generator=subkey_generator_default)
        B = np.hstack((B, encrypted_value.reshape(-1, 1)))


    #print(B)

    return A, B

In [12]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

A, B = linear_relationship()

#print(A)
#print(B)
print((np.dot(A,k) + np.dot(B,u))%p)
print(np.all(x == (np.dot(A,k) + np.dot(B,u))%p))

[4 0 0 9 7 0 0 3]
True


In [13]:
u_1 = np.array([0,0,0,0,0,0,0,0]) #change plaintext to test
k_1 = np.array([0,0,0,0,0,0,0,1]) #change key to test

x = encryption(u_1,k_1,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

A, B = linear_relationship()

print(A)
print(B)

print(x)
print((np.dot(A,k_1) + np.dot(B,u_1))%p)

print(x == (np.dot(A,k_1) + np.dot(B,u_1))%p)

[[ 9  0  1  6  0  0  1 10]
 [ 0  8  6  2  2  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  8  0  1  6  6]
 [ 2  0  1 10  0  0  1  3]
 [ 0  1  8  4  9  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  1  0  1  4  8]]
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
[10  0  0  6  3  0  0  8]
[10  0  0  6  3  0  0  8]
[ True  True  True  True  True  True  True  True]


## Task 4
From a known plaintext/ciphertext pair $(u, x)$, implement a linear cryptanalysis KPA against this cipher by computing 
$$k = A^{−1}(x − Bu) \mod p$$ 
with all operations in the field $\mathbb{F} = GF(p)$

In [43]:
determinant = round(np.linalg.det(A))
A_ast = np.linalg.inv(A)

A_tilda = A_ast*determinant

det_inv = pow(determinant, -1, p)

A_inv = np.round(A_tilda *det_inv).astype(int) % p

print(A_inv)



[[10  9  6 10  2  0  9  4]
 [ 8  0  9  4  4  4  2  2]
 [ 3 10  9  7  7  1  3  9]
 [ 0  2  7  0  0  4  7  0]
 [ 2  5  8  1  1  0  0  6]
 [10  9  1  5  5  1  2  8]
 [ 5  7  3  3  9  4  1  6]
 [ 9  0  9  1  1  7  6  7]]


In [49]:
u = u_1
print(u_1)
x = np.array([10, 0, 0, 6, 3, 0, 0, 8])
print(x)
key = (A_inv@(x - B@u)) %p

print(key)


[0 0 0 0 0 0 0 0]
[10  0  0  6  3  0  0  8]
[0 0 0 0 0 0 0 1]


## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the substitution function described by the following table
|$v_i(j)$|0|1|2|3|4|5|6|7|8|9|10|
|--------|-|-|-|-|-|-|-|-|-|-|--|
|$y_i(j)$|0|2|4|8|6|10|1|3|5|7|9|

where $j ∈ \{1, . . . , ℓ\}$

In [ ]:
def custom_substitution_number(v):
    if(v == 0):
        return 0
    if(v == 1):
        return 2
    if(v == 2):
        return 4
    if(v == 3):
        return 8
    if(v == 4):
        return 6
    if(v == 5):
        return 10
    if(v == 6):
        return 1
    if(v == 7):
        return 3
    if(v == 8):
        return 5
    if(v == 9):
        return 7
    if(v == 10):
        return 9
    
def custom_substitution(v,p):

    for i in range(8):
        v[i] = custom_substitution_number(v[i])
    
    return v


In [ ]:
print(encryption(u,k,p,r,
                 substitution=custom_substitution,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator=subkey_generator_default
                 ))

[9 8 0 7 6 2 7 9]


## Task 6
Linear cryptanalysis of a “nearly linear” cipher

## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters: $\mathcal{K} = \mathbb{F}^{\ell_k}, \ell_k = 4$

ECC...

In [ ]:
def subkey_generator_task_7(key):

    k_1 = [key[0],key[1],key[2],key[3]]
    k_2 = [key[0],key[1],key[3],key[2]]
    k_3 = [key[1],key[2],key[3],key[0]]
    k_4 = [key[0],key[3],key[1],key[2]]
    k_5 = [key[2],key[3],key[0],key[1]]
    k_6 = [key[1],key[3],key[0],key[2]]

    return [k_1,k_2,k_3,k_4,k_5,k_6]

def mod_inv(v, p):
    
    x =[]

    for num in v:
        if(num != 0):
            x.append(2*pow(num, -1, p))
        else:
            x.append(0)


    return np.array(x) # Calcola l'inverso di x modulo p

def substitution_task_7(x,p):
    x= x.tolist()
    return mod_inv(x, p)
    

In [ ]:
u= np.array([1,0,0,0,0,0,0,0])
k=np.array([1,0,0,0])

p= 11

print(encryption(u,k,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7))

[5 0 3 2 5 2 1 1]
